In [ ]:
import io
import csv
import string
import pandas as pd
import sklearn
import numpy as np
!apt-get install graphviz
!pip install dtreeviz

from sklearn.datasets import *
from dtreeviz.trees import *
from IPython.core.display import display, HTML
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.datasets import make_hastie_10_2

from sklearn.tree import DecisionTreeClassifier # Import Decision Tree Classifier
from sklearn.model_selection import train_test_split # Import train_test_split function
from sklearn import metrics #Import scikit-learn metrics module for accuracy calculation

In [ ]:
file = open("/content/drive/MyDrive/Data Mining/HW2_credit_train.csv","r")
col_names = ['id','yag','workhrs','val','occ','gain','loss','ms','emptyp','edytyp','race','gender','credit']
trainingData = pd.read_csv(file)

trainingData =  pd.DataFrame(trainingData)
print(trainingData.head(0))
trainingData.columns = col_names
print(trainingData.columns)
print(trainingData)
#print(trainingData.head(0))
#print(trainingData.info())
#trainingData["credit"].plot(kind = 'hist')

In [ ]:
file = open("/content/drive/MyDrive/Data Mining/HW2_credit_test.csv","r")
col_names = ['id','yag','workhrs','val','occ','gain','loss','ms','emptyp','edytyp','race','gender']
testData = pd.read_csv(file)

testData =  pd.DataFrame(testData)
testData.columns = col_names
print(testData)


In [ ]:
trainingData.info()

In [ ]:
trainingData.emptyp.value_counts()


In [ ]:
trainingData.edytyp.value_counts()


In [ ]:
trainingData.race.value_counts()


In [ ]:
trainingData.gender.value_counts()


In [ ]:
trainingData.val.value_counts()


In [ ]:
trainingData.boxplot('emptyp','credit',rot = 30,figsize=(5,6))

In [4]:
feature_cols = ['yag','workhrs','val','occ','gain','loss','ms','emptyp','edytyp'] ## here id , gender and race have not been used as features
X = trainingData[feature_cols]
y = trainingData.credit

X_train,X_test,y_train,y_test = train_test_split(X, y, test_size=0.3, random_state=1)


In [126]:
print(feature_cols)

['yag', 'workhrs', 'val', 'occ', 'gain', 'loss', 'ms', 'emptyp', 'edytyp']


In [ ]:
## though the other variables like emptyp,edytyp , ms , occ , val are all supposed to be categorical they are of the datatype int64 
#and have already been normalised and hence their normalisation has not been considered here
#one-hot encoding for categorical features gender and race is being checked here
cat_features = trainingData.select_dtypes(include=['object']).copy() 
print(cat_features)

from sklearn.preprocessing import LabelBinarizer

lb_race = LabelBinarizer()
lb_results_race = lb_race.fit_transform(cat_features['race'])
lb_results_df_race = pd.DataFrame(lb_results_race, columns=lb_race.classes_)

print(lb_results_df_race.head())


In [37]:
def tree_graph_to_png(tree, feature_names, png_file_to_save):
    tree_str = export_graphviz(tree, feature_names=feature_names, 
                                     filled=True, out_file=None)
    graph = pydotplus.graph_from_dot_data(tree_str)  
    graph.write_png(png_file_to_save)

In [143]:
def decision_tree_class(X_train,y_train,X_test):
  decision_clssf = DecisionTreeClassifier(max_depth=3)
  decision_clssf = decision_clssf.fit(X_train,y_train)
  y_pred = decision_clssf.predict(X_test)
  tree_graph_to_png(decision_clssf,feature_cols,"tree")
  return y_pred

In [137]:
def decision_tree_class_entropy(X_train,y_train,X_test):
  decision_clssf = DecisionTreeClassifier(criterion="entropy", max_depth=3)
  decision_clssf = decision_clssf.fit(X_train,y_train)
  y_pred_entropy = decision_clssf.predict(X_test)
  tree_graph_to_png(decision_clssf,feature_cols,"tree1")
  return y_pred_entropy

In [95]:
def decision_tree_boost(X_train,y_train,X_test):
  tree_boost = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0,
     max_depth=1, random_state=0).fit(X_train, y_train)
  y_pred_boost = tree_boost.predict(X_test)
  tree_graph_to_png(tree_boost,feature_cols,"tree2")
  return y_pred_boost

In [ ]:
from imblearn.over_sampling import SMOTE, ADASYN
from collections import Counter
from sklearn.svm import LinearSVC, SVC

X_resampled, y_resampled = SMOTE().fit_resample(X, y)
print(sorted(Counter(y_resampled).items()))
clf_smote = LinearSVC().fit(X_resampled, y_resampled)
X_resampled, y_resampled = ADASYN().fit_resample(X, y)
print(sorted(Counter(y_resampled).items()))
clf_adasyn = LinearSVC().fit(X_resampled, y_resampled)

In [135]:
from sklearn.metrics import f1_score

f1_score(y_test, y_pred, average='macro')


0.7701651185311991

In [136]:
f1_score(y_test, y_pred_entropy, average='macro')

0.7690819227130525

In [145]:
f1_score(y_test, y_pred_os, average='macro')

0.7078638911735582

In [142]:
f1_score(y_test, y_pred_entropy_os, average='macro')

0.6577629649180569

In [138]:
y_pred = decision_tree_class(X_train,y_train,X_test)
y_pred_entropy = decision_tree_class_entropy(X_train,y_train,X_test)

print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
print("Accuracy:",metrics.accuracy_score(y_test, y_pred_entropy))


dot: graph is too large for cairo-renderer bitmaps. Scaling by 0.28992 to fit

Accuracy: 0.8439963148735797
Accuracy: 0.8458388780837343


In [144]:
y_pred_os = decision_tree_class(X_resampled,y_resampled,X_test)
y_pred_entropy_os = decision_tree_class_entropy(X_resampled,y_resampled,X_test)

print("Accuracy:",metrics.accuracy_score(y_test, y_pred_os))
print("Accuracy:",metrics.accuracy_score(y_test, y_pred_entropy_os))

Accuracy: 0.7492066741734057
Accuracy: 0.683693315590132


In [ ]:
test_X = testData[feature_cols]
result = decision_tree_class(X,y,test_X)
print(result)

In [120]:
test_X = testData[feature_cols]
result_entropy = decision_tree_class(X,y,test_X)
print(result_entropy)

[0 1 1 ... 0 1 0]


In [146]:
test_X = testData[feature_cols]
result_os = decision_tree_class(X_resampled,y_resampled,test_X)
print(result_os)

[0 1 1 ... 0 1 0]


In [122]:
test_X = testData[feature_cols]
result_entropy_os = decision_tree_class_entropy(X_resampled,y_resampled,test_X)
print(result_entropy_os)

[0 1 1 ... 0 1 0]


In [22]:
## result file created here . Run this for test data run
f = open("result_decision_tree.txt", "w")
for i in list(result):
    if i==1:
        f.write("1\n")
    else:
        f.write("0\n")
f.close()

In [114]:
## result file created here . Run this for test data run
f = open("result_decision_tree_entropy.txt", "w")
for i in list(result_entropy):
    if i==1:
        f.write("1\n")
    else:
        f.write("0\n")
f.close()

In [147]:
## result file created here . Run this for test data run
f = open("result_decision_tree__os.txt", "w")
for i in list(result_os):
    if i==1:
        f.write("1\n")
    else:
        f.write("0\n")
f.close()

In [148]:
## result file created here . Run this for test data run
f = open("result_decision_tree__entropy_os.txt", "w")
for i in list(result_entropy_os):
    if i==1:
        f.write("1\n")
    else:
        f.write("0\n")
f.close()